In [21]:
import pandas as pd
import tabula as tb
from tabula import read_pdf

In [22]:
paths_pdf = [r"C:\Users\\adria\Desktop\Miratoni/Imports/July/23-7-21/Main Lorry/Despacho FAC A21_623 _ 627.pdf"]
paths_csv = [r"C:\Users\\adria\Desktop\Miratoni/Imports/July/23-7-21/Main Lorry/Despacho FAC A21_623 _ 627.csv"]
pd.set_option("max_colwidth", 1000)
pd.set_option("display.max_rows", 200)

desc_areas = [[146.891,303.45,156.559,574.175],[215.316,303.45,227.216,574.919],[285.228,303.45,296.384,574.175],[355.884,304.194,366.297,574.919],[425.797,303.45,437.697,575.663],[495.709,302.706,506.866,574.919],[566.366,304.194,577.522,574.175],[636.278,303.45,646.691,573.431],[706.191,304.194,716.603,574.919]]
code_areas = [[165.484,304.194,177.384,574.175],[235.397,303.45,246.553,574.175],[306.053,304.194,316.466,574.175],[375.222,303.45,387.866,574.919],[445.878,304.194,456.291,574.919],[516.534,303.45,526.947,575.663],[584.959,303.45,596.116,574.919],[656.359,304.194,666.772,574.919],[726.272,304.194,735.941,574.175]]
gr_wt_areas = [[196.722,490.875,206.391,574.919],[266.634,490.131,277.047,574.175],[335.803,490.875,346.959,574.919],[405.716,489.388,417.616,574.175],[475.628,490.875,486.784,574.175],[545.541,490.875,557.441,574.919],[615.453,490.875,627.353,574.919],[686.109,490.875,697.266,574.175],[756.022,490.875,765.691,574.919]]
net_wt_areas = [[206.391,490.875,216.803,574.175],[275.559,490.875,287.459,574.175],[346.216,490.875,356.628,574.175],[416.128,490.875,426.541,573.431],[486.041,490.875,496.453,574.175],[555.953,490.875,566.366,574.175],[626.609,490.875,636.278,574.919],[695.778,490.875,706.934,573.431],[765.691,490.875,775.359,574.175]]
value_areas = [[205.647,375.594,215.316,491.619],[275.559,375.594,285.972,490.875],[346.216,375.594,355.141,492.363],[416.128,375.594,425.053,492.363],[486.784,375.594,497.197,491.619],[555.953,375.594,566.366,491.619],[625.866,375.594,637.022,491.619],[696.522,375.594,705.447,490.619],[766.434,375.594,776.847,492.363]]
first_page_areas = [[487.528,298.988,499.428,574.175],[509.841,296.756,520.253,574.919],[539.591,487.156,550.003,573.431],[549.259,486.413,560.416,574.175],[549.259,374.85,560.416,486.413]]
normal_areas = [134.991, 303.078, 778.334, 576.778]


In [43]:
desc = tb.read_pdf(paths_pdf[0], pages="all", stream=True, area=desc_areas,
                 multiple_tables=False, format="CSV")
commcode = tb.read_pdf(paths_pdf[0], pages="all", stream=True, area=code_areas,
                 multiple_tables=False, format="CSV")
gr_wgt = tb.read_pdf(paths_pdf[0], pages="all", stream=True, area=gr_wt_areas,
                 multiple_tables=False, format="CSV")
nt_wgt = tb.read_pdf(paths_pdf[0], pages="all", stream=True, area=net_wt_areas,
                 multiple_tables=False, format="CSV")
values = tb.read_pdf(paths_pdf[0], pages="all", stream=True, area=value_areas,
                 multiple_tables=False, format="CSV")
print(nt_wgt)
#df2 = pd.concat(df,axis=1,ignore_index=True)


[       140.0
0    810.109
1     63.192
2      2.000
3      6.000
4      3.600
5      4.000
6      0.480
7      6.000
8      0.960
9    120.000
10   256.000
11   120.000
12   162.000
13   158.120
14    60.000
15     6.000
16    57.480
17    38.400
18  1044.000
19   178.020
20   411.960
21    10.220
22   153.600
23   186.400
24    23.200
25    11.400
26    24.000
27   106.864
28     8.100
29   268.800
30    16.272
31     8.800
32    42.336
33   366.384
34    38.400
35     3.300
36   321.492
37   543.200
38    64.800
39   202.500
40   315.000
41   171.000
42   135.000
43    90.000
44   432.000
45  1080.000
46    30.000
47     6.000
48     1.440
49   189.721
50    12.000]


In [46]:
df = pd.DataFrame.from_dict({'Description': desc,'PTCode':commcode,'Gross Weight (kg)':gr_wgt,'Net Weight(kg)':nt_wgt,'Value (EUR)':values}, orient='columns')
print(df)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Descripti

In [11]:


first_page = tb.read_pdf(paths_pdf[0], pages=1, stream=True, area=first_page_areas,
                         options="--columns 415,521,675,798")
df[0] = first_page[0]

In [ ]:
def create_table(path):

In [9]:
all_df = pd.concat(df, axis=0, ignore_index=True)
all_df = all_df.dropna(how='all')

print(all_df)

    Tipo de declaração (1)   Valor Estatístico (4   6)   Massa líquida  \
0                             BACALHAU EM PEDACOS                  NaN   
1                                         0304952  5.00.0000.0000.0000   
2                                         1000---                  NaN   
3                                             NaN                184.2   
4                                             NaN                  NaN   
..                                            ...                  ...   
207                                           NaN                  NaN   
208                                           NaN                  NaN   
209                                           NaN                  NaN   
210                                           NaN                  NaN   
211                                           NaN                  NaN   

     (kg) (38) Tipo de declaração (1)   Valor Estatístic  \
0          NaN                                     

In [ ]:
all_df.to_csv("C:\Users\adria\Documents\Dev\Customs Note Reader\customs note.csv")